# Set up

In [4595]:
from worde4mde import load_embeddings

sgram_mde = load_embeddings("sgram-mde")

In [4596]:
word = "ecore"
sgram_mde[word]

array([ 0.14674647,  0.42704162,  0.17717203,  0.05179158,  0.38020504,
       -0.00091264,  0.02094162,  0.00924711, -0.13806985,  0.1403339 ,
       -0.0953936 , -0.03681302,  0.38122246, -0.03746711, -0.17595871,
       -0.02031669,  0.23675247,  0.27676958, -0.07487358, -0.1987225 ,
        0.02015282,  0.05147164, -0.223504  ,  0.00699752, -0.06145973,
        0.00920826,  0.01364714,  0.10218145, -0.25330105, -0.14080754,
        0.2541444 ,  0.4577827 , -0.16825216, -0.08050938,  0.11404785,
       -0.38822263, -0.06395938, -0.4724689 , -0.00111705,  0.27869806,
        0.28032073,  0.02976846, -0.01675524,  0.09774698,  0.18298927,
       -0.24398401, -0.3238438 ,  0.01912955,  0.11861664,  0.08640955,
        0.18799016, -0.06891091, -0.03893414, -0.04862301,  0.02151174,
       -0.39205688, -0.23195332, -0.09532862, -0.01324714, -0.24555108,
        0.09372689, -0.21971853, -0.26171875, -0.3605202 , -0.07306654,
       -0.16288763,  0.12160088,  0.11416717,  0.17378834, -0.04

In [4597]:
# import pickle

import ast
import pandas as pd
import numpy as np

from worde4mde import load_embeddings
import re

import os
import openai
from openai import OpenAI
import json

import networkx as nx
import sys

In [4598]:
from dotenv import load_dotenv

# %%

load_dotenv()

openai_key = os.getenv("openai_key")
os.environ["OPENAI_API_KEY"] = openai_key

## Input

In [ ]:
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######

instructor_dir = "results"

# 01_LabTracker
# 02_CelO
# 03_TSS
# 04_SHAS
# 05_OTS
# 06_Block
# 07_Tile
# 08_HBMS
problem_id = "03_TSS"

# location of reference domain model is stored. It is in .txt format
instructor_input = instructor_dir "/" + problem_id + ".txt"

# gpt-4o-mini
# gpt-4
# gpt-4o
llm_id = "gpt-4"

# yujing
# sherpa
method_id = "sherpa"

student_dir = instructor_dir + '/' + llm_id + '/' + method_id + '/' + problem_id + '/'
if not os.path.exists(student_dir):
    # Create the directory
    os.makedirs(student_dir)
    
# location of generated domain model is saved. The file should be in .txt    
student_input = student_dir + problem_id + ".txt"
student_input
student_out_dir = student_dir + "result/"
student_out_dir

'results/gpt-4/sherpa/03_TSS/result/'

In [4600]:
# First of all, you need to load the embeddings (currently supported: 'sgram-mde' and 'glove-mde')
sgram_mde = load_embeddings("sgram-mde")
# glove = load_embeddings('glove-mde')
# sgram_mde["id"]

In [4601]:
# word = 'id'
# sgram_mde.most_similar(positive=[word])

In [4602]:
def inEmbedding(word, embedding):
    try:
        return True, embedding[word]
    except:
        return False, None

In [4603]:
def splitCamelCase(word):
    splitted = re.sub("([A-Z][a-z]+)", r" \1", re.sub("([A-Z]+)", r" \1", word)).split()
    return splitted


splitCamelCase("device ID")

['device', 'ID']

In [4604]:
splitCamelCase("CommandStatus")

['Command', 'Status']

In [4605]:
# client = OpenAI()

# def run_llm(prompt, model ="gpt-3.5-turbo"):
#   response = client.chat.completions.create(
#     model=model,
#     messages=[
#       {"role": "user", "content": prompt},

#     ]
#   )
#   return response.choices[0].message.content

In [4606]:
client = OpenAI()


def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [4607]:
def get_all_info(class_index, class_nodes, list_of_classes, list_edges, need_edge=True):
    result = ""
    result += list_of_classes[class_index] + "\n"
    node = class_nodes[class_index]
    if "abstract" in node:
        node = node.replace("abstract", "").strip()
    if "Abstract" in node:
        node = node.replace("Abstract", "").strip()
    if need_edge:
        for edge in list_edges:
            element = [i.strip() for i in edge.split()]
            if node in element:
                result += edge + "\n"

    return result

In [4608]:
def match_classes(raw_1, raw_2, dict_attr, thresh=0.5, verbose=False):
    # raw_1: nodes from reference solution
    # raw_2: nodes from student solution
    # dict_attr: dictionary for embeddings
    # thresh for subst cost
    # map class lists raw_1 and raw_2
    print(dict_attr)

    def node_subst_cost_attr(node1, node2):
        # threshod as 0.45
        if dict_attr[node1["name"]][node2["name"]] < thresh:
            return 3
        else:
            return 1 - dict_attr[node1["name"]][node2["name"]]

    G_att_1 = nx.Graph()
    G_att_2 = nx.Graph()
    for node in raw_1:
        G_att_1.add_node(node, name=node)

    for node in raw_2:
        G_att_2.add_node(node, name=node)

    for v in nx.optimize_edit_paths(
        G_att_1,
        G_att_2,
        node_subst_cost=node_subst_cost_attr,
        edge_match=None,
        timeout=20,
    ):
        minv = v
    # minv

    return minv

# Group

In [4609]:
class Grader:
    def __init__(self):
        self.ref = None
        self.stu = None

In [4610]:
grader = Grader()

## Class

### Define metadata

In [4611]:
class ModelInfo:

    def __init(self):
        self.cls_atr = {}
        self.cls_name = []
        self.raw_dsl = []
        self.rel = None

In [4612]:
ref_cls = ModelInfo()
stu_cls = ModelInfo()

In [4613]:
grader.ref = ref_cls
grader.stu = stu_cls

In [4614]:
with open(instructor_input, "r") as file:
    # Read the entire content of the file
    instructor_all = file.read()

with open(student_input, "r") as file:
    # Read the entire content of the file
    student_all = file.read()

In [4615]:
print(instructor_all)

Enumerations:
Position(GK, LB)
Recommendation(KEY_PLAYER, FIRST_TEAM_PLAYER, RESERVE_TEAM_PLAYER, PROSPECTIVE_PLAYER, NOT_RECOMMENDED)
ScoutingStatus(LONG_LISTED, SHORT_LISTED, RECOMMENDED_FOR_SIGNING, OFFER_MADE)
ScoutKind(REGULAR_SCOUNT, HEAD_SCOUT)


Classes:
TSS()
abstract Person(string firstName, string lastName)
Player(ScoutingStatus status)
HeadCoach()
Director()
Scout(ScoutKind scoutKind)
Club()
Offer(int value)
ScoutingAssignmnet()
ScoutReport(string pro, string con, Recommendation recommendation)
PlayerProfile(Position position)
PlayerAttribute(string name, int value)


Relationships:



In [4616]:
print(student_all)

Enumerations:
Recommendation(KeyPlayer, FirstTeamPlayer, ReserveTeamPlayer, ProspectivePlayer, NotAGoodSigning)


Classes:
TeamSportsScoutingSystem(string systemName)
Employee(string employeeName)
Player(string playerName, string playerAttributes)
Director(string directorName)

Scout(string scoutName) 
Club(string clubName)
Offer(string offerName)
ScoutingAssignment(string assignmentName)
ScoutingReport(string reportName, string pros, string cons, string recommendation)
PlayerProfile(string profileName)
PlayerAttribute(string attributeName, string attributeValue)
TargetPosition(string positionName)
LongList(string longListName)
HeadScout(string headScoutName)

ShortList(string shortListName)

Relationships:



In [4617]:
shas_sol = instructor_all.split("Relationships:")[0]
stu_sol_g = student_all.split("Relationships:")[0]  # student solution

In [4618]:
ref_classes_raw = []
tmp = shas_sol.strip().splitlines()
ref_classes_raw = [i.strip() for i in tmp if len(i) > 0]
ref_classes_raw

reference_class = []
tmp = shas_sol.strip().splitlines()
reference_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]

In [4619]:
stu_classes_raw = []
tmp = stu_sol_g.strip().splitlines()
stu_classes_raw = [i.strip() for i in tmp if len(i) > 0]
stu_classes_raw

tmp = stu_sol_g.strip().splitlines()
stu_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]
stu_class

['Enumerations:',
 'Recommendation',
 'Classes:',
 'TeamSportsScoutingSystem',
 'Employee',
 'Player',
 'Director',
 'Scout',
 'Club',
 'Offer',
 'ScoutingAssignment',
 'ScoutingReport',
 'PlayerProfile',
 'PlayerAttribute',
 'TargetPosition',
 'LongList',
 'HeadScout',
 'ShortList']

In [4620]:
ref_attributes = {}
enum_index = ref_classes_raw.index("Enumerations:")
regular_index = ref_classes_raw.index("Classes:")
for cla, dsl in zip(reference_class, ref_classes_raw):
    if dsl == "Enumerations:" or dsl == "Classes:":
        continue
    else:
        index = ref_classes_raw.index(dsl)
        if index > regular_index:
            class_type = "regular"
            if "abstract" in dsl:
                class_type = "abstract"
        else:
            class_type = "enum"
        ref_attributes[cla] = {
            "score": 0,
            "type": class_type,
            "dsl": dsl,
            "counterpart": None,
            "attributes": {},
        }

        attributes = dsl.split("(")[1][:-1].split(",")
        for attr in attributes:
            attr = attr.strip()
            if len(attr) > 0:
                ref_attributes[cla]["attributes"][attr] = {
                    "score": 0,
                    "counterpart": None,
                }

In [4621]:
reference_class.remove("Enumerations:")
reference_class.remove("Classes:")

ref_classes_raw.remove("Enumerations:")
ref_classes_raw.remove("Classes:")

In [4622]:
ref_cls.cls_name = reference_class
ref_cls.raw_dsl = ref_classes_raw
ref_cls.cls_atr = ref_attributes

In [4623]:
stu_attributes = {}
enum_index = stu_classes_raw.index("Enumerations:")
regular_index = stu_classes_raw.index("Classes:")

for cla, dsl in zip(stu_class, stu_classes_raw):
    if dsl == "Enumerations:" or dsl == "Classes:":
        continue
    else:
        index = stu_classes_raw.index(dsl)
        if index > regular_index:
            class_type = "regular"
            if "abstract" in dsl:
                class_type = "abstract"
        else:
            class_type = "enum"

        stu_attributes[cla] = {
            "score": 0,
            "type": class_type,
            "dsl": dsl,
            "counterpart": None,
            "attributes": {},
        }

        print(dsl)
        attributes = dsl.split("(")[1][:-1].split(",")
        for attr in attributes:
            attr = attr.strip()
            if len(attr) > 0:
                stu_attributes[cla]["attributes"][attr] = {
                    "score": 0,
                    "counterpart": None,
                }

Recommendation(KeyPlayer, FirstTeamPlayer, ReserveTeamPlayer, ProspectivePlayer, NotAGoodSigning)
TeamSportsScoutingSystem(string systemName)
Employee(string employeeName)
Player(string playerName, string playerAttributes)
Director(string directorName)
Scout(string scoutName)
Club(string clubName)
Offer(string offerName)
ScoutingAssignment(string assignmentName)
ScoutingReport(string reportName, string pros, string cons, string recommendation)
PlayerProfile(string profileName)
PlayerAttribute(string attributeName, string attributeValue)
TargetPosition(string positionName)
LongList(string longListName)
HeadScout(string headScoutName)
ShortList(string shortListName)


In [4624]:
stu_class.remove("Enumerations:")
stu_class.remove("Classes:")

stu_classes_raw.remove("Enumerations:")
stu_classes_raw.remove("Classes:")

In [4625]:
stu_cls.cls_name = stu_class
stu_cls.raw_dsl = stu_classes_raw
stu_cls.cls_atr = stu_attributes

## Edges

### reference and student solutions

In [4626]:
shas_edges = instructor_all.split("Relationships:")[1]
shas_G2_edges = student_all.split("Relationships:")[1]  # student solution

In [4627]:
shas_edges

'\n'

In [4628]:
shas_edges

'\n'

In [4629]:
shas_edges = shas_edges.replace("0..*", "*")

In [4630]:
shas_G2_edges = shas_G2_edges.replace("0..*", "*")

In [4631]:
tmp = shas_edges.strip().splitlines()
ref_edges = [i.strip() for i in tmp if len(i) > 0]

for i in ref_edges:
    length = len(i.split())
    if length == 5 or length == 3:
        pass
    else:
        print("Reference Length error:", i)

tmp = shas_G2_edges.strip().splitlines()
stu_edges = [i.strip() for i in tmp if len(i) > 0]

for i in stu_edges:
    length = len(i.split())
    if length == 5 or length == 3:
        pass
    else:
        print("Student Length error:", i)

In [4632]:
class Relationship:
    def __init__(self):
        self.rels = []  # list of dict{}
        self.raw_dsl = []

    # find first non-matched or matched reltionshipi given dsl
    def find_relation(self, dsl, matched=False):
        for index, i in enumerate(self.rels):
            if i["dsl"] == dsl and i["counterpart"] is None:
                return index, i

        return None

In [4633]:
ref_edges_obj = Relationship()
stu_edges_obj = Relationship()


ref_edges_dict = []
for e in ref_edges:
    ref_edges_dict.append({"dsl": e, "score": 0, "counterpart": None})

ref_edges_obj.rels = ref_edges_dict
ref_edges_obj.raw_dsl = ref_edges

stu_edges_dict = []
for e in stu_edges:
    stu_edges_dict.append({"dsl": e, "score": 0, "counterpart": None})

stu_edges_obj.rels = stu_edges_dict
stu_edges_obj.raw_dsl = stu_edges

In [4634]:
# link to classInfo object
ref_cls.rel = ref_edges_obj
stu_cls.rel = stu_edges_obj

In [4635]:
edges = [ref_edges_obj, stu_edges_obj]

## Getting embedding

In [4636]:
def removeChars(text, chars):
    for c in chars:
        text = text.replace(c, "")
    return text

In [4637]:
def get_mde_embedding(text, embedding):
    # >>> get_mde_embedding("WhatDevice", sgram_mde)
    words = splitCamelCase(removeChars(text, ["(", ")", ","]))
    lowercase_list = [s.lower() for s in words]
    counter = 0
    emb = np.zeros(300)
    for w in lowercase_list:
        try:
            emb += embedding[w]
            counter += 1
        except:
            # if failed in emb the complete word, embed the token
            tmp = np.zeros(300)
            for char in list(w):

                tmp += embedding[char]
            emb += tmp / len(w)
            counter += 1
    return emb / counter

In [4638]:
text = "AutomationStatus (Created, Edited, Activated, Deactivated)"
words = splitCamelCase(removeChars(text, ["(", ")", ","]))
words

['Automation', 'Status', 'Created', 'Edited', 'Activated', 'Deactivated']

In [4639]:
def cosine_distance(emb_i, emb_j):
    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

## Stage 1.1 Class name mapping

In [4640]:
# get name embedding
# all info
similarity_mde = []
mde_embedding = sgram_mde
threshold = 0.7
similarity_mde_dsl = []
percentage = 0.8
for index, node in enumerate(ref_cls.cls_name):
    cls = node.split()[-1].strip()  # get the class name, remove abstract key word

    mde_emb_i = get_mde_embedding(cls, mde_embedding)
    mde_emb_i_dsl = get_mde_embedding(ref_cls.cls_atr[node]["dsl"], mde_embedding)

    pair = []
    mde_pair = []
    for j, stu_node in enumerate(stu_cls.cls_name):
        cls = stu_node.split()[
            -1
        ].strip()  # get the class name, remove abstract key word
        print()
        mde_emb_j = get_mde_embedding(cls, mde_embedding)
        mde_emb_j_dsl = get_mde_embedding(
            stu_cls.cls_atr[stu_node]["dsl"], mde_embedding
        )

        mde_sim = cosine_distance(mde_emb_i, mde_emb_j)
        mde_sim_dsl = cosine_distance(mde_emb_i_dsl, mde_emb_j_dsl)

        mde_pair.append(mde_sim * percentage + (1 - percentage) * mde_sim_dsl)

    # apply third quartile
    similarity_mde.append(mde_pair)

dict_sim_word = {}
for i in range(len(ref_cls.cls_name)):
    dict_sim_word[ref_cls.cls_name[i]] = {}
    for j in range(len(stu_cls.cls_name)):
        dict_sim_word[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_mde[i][j]

In [4641]:
for key in dict_sim_word:
    exact_match = False
    exact_cls = ""
    for cls_2 in dict_sim_word[key]:
        if dict_sim_word[key][cls_2] > 0.99:
            exact_match = True
            exact_cls = cls_2
            print(cls_2)
            break

    if exact_match:  # keep the exact match, others to 0
        print("update")
        for cls_2 in dict_sim_word[key]:
            if cls_2 != exact_cls:
                dict_sim_word[key][cls_2] = 0

similarity_mde = []
for i in dict_sim_word:
    tmp = []
    for j in dict_sim_word[i]:
        tmp.append(dict_sim_word[i][j])

    similarity_mde.append(tmp)

In [4642]:
import pandas as pd

df = pd.DataFrame(index=reference_class, columns=stu_class, data=similarity_mde)
df

,Recommendation,TeamSportsScoutingSystem,Employee,Player,Director,Scout,Club,Offer,ScoutingAssignment,ScoutingReport,PlayerProfile,PlayerAttribute,TargetPosition,LongList,HeadScout,ShortList
Position,0.172897,0.245518,0.256960,0.280882,0.201255,0.147635,0.203183,0.125496,0.216780,0.252673,0.290412,0.300761,0.736684,0.221491,0.316544,0.257041
Recommendation,0.946761,0.284232,0.156030,0.210642,0.099612,0.163609,0.213497,0.245176,0.199961,0.227239,0.290341,0.198332,0.218656,0.239719,0.167116,0.238273
ScoutingStatus,0.154948,0.448950,0.273992,0.230127,0.301752,0.606249,0.144908,0.289805,0.523588,0.524175,0.292092,0.315576,0.397024,0.303973,0.432758,0.296289
ScoutKind,0.121998,0.409288,0.242748,0.150650,0.219351,0.754001,0.209097,0.322863,0.569662,0.496742,0.258636,0.266970,0.326645,0.205059,0.480803,0.211907
TSS,0.068208,0.216140,0.112344,0.072561,0.160169,0.143910,0.077671,0.071599,0.143136,0.229334,0.081298,0.061744,0.109590,0.194986,0.188797,0.162323
abstract Person,0.117581,0.372074,0.629815,0.385744,0.322782,0.263277,0.373273,0.198894,0.291558,0.264242,0.391945,0.471068,0.405813,0.253761,0.302885,0.211176
Player,0.178987,0.327150,0.252914,0.935426,0.285407,0.123244,0.289916,0.179970,0.229124,0.152251,0.777736,0.819377,0.283653,0.199848,0.195694,0.191571
HeadCoach,0.203141,0.393395,0.358551,0.300330,0.358061,0.135832,0.217497,0.092529,0.172208,0.247091,0.315448,0.319593,0.294440,0.262622,0.609640,0.256676
Director,0.063904,0.298537,0.271992,0.240883,0.957426,0.100465,0.172412,0.124772,0.154469,0.160314,0.232441,0.250068,0.159880,0.189285,0.334501,0.172234
Scout,0.091547,0.426460,0.218074,0.114783,0.173729,0.957523,0.140372,0.218839,0.674127,0.581620,0.183469,0.210617,0.275403,0.160172,0.590252,0.174804


### Stage 1.1.1 Mapping (enum / regular + abstract)

In [4643]:
ref_enum = []
ref_reg_cls = []
stu_enum = []
stu_reg_cls = []

for enum in ref_cls.cls_name:
    if ref_cls.cls_atr[enum]["type"] == "enum":
        ref_enum.append(enum)
    else:
        ref_reg_cls.append(enum)

for enum in stu_cls.cls_name:
    if stu_cls.cls_atr[enum]["type"] == "enum":
        stu_enum.append(enum)
    else:
        stu_reg_cls.append(enum)

In [4644]:
enum_mapping = match_classes(ref_enum, stu_enum, dict_sim_word, thresh=0.5)
enum_mapping[0]

{'Position': {'Recommendation': 0.17289720701248845, 'TeamSportsScoutingSystem': 0.24551773111691017, 'Employee': 0.2569598471529636, 'Player': 0.2808824355394257, 'Director': 0.2012549174947853, 'Scout': 0.14763478806961922, 'Club': 0.20318341445028992, 'Offer': 0.1254959967663124, 'ScoutingAssignment': 0.21678038572140362, 'ScoutingReport': 0.25267279289563177, 'PlayerProfile': 0.2904123917938123, 'PlayerAttribute': 0.30076061345532484, 'TargetPosition': 0.7366842964305249, 'LongList': 0.22149131081650525, 'HeadScout': 0.3165436764150311, 'ShortList': 0.25704129978623275}, 'Recommendation': {'Recommendation': 0.9467610253735672, 'TeamSportsScoutingSystem': 0.28423185844541937, 'Employee': 0.15602987508633503, 'Player': 0.21064163184993834, 'Director': 0.09961178185162307, 'Scout': 0.16360857301047652, 'Club': 0.21349687448651072, 'Offer': 0.24517642622027988, 'ScoutingAssignment': 0.19996121414834356, 'ScoutingReport': 0.22723861441753263, 'PlayerProfile': 0.2903410213572166, 'Player

[('Position', None),
 ('Recommendation', 'Recommendation'),
 ('ScoutingStatus', None),
 ('ScoutKind', None)]

In [4645]:
cls_mapping = match_classes(ref_reg_cls, stu_reg_cls, dict_sim_word, thresh=0.5)
cls_mapping[0]

{'Position': {'Recommendation': 0.17289720701248845, 'TeamSportsScoutingSystem': 0.24551773111691017, 'Employee': 0.2569598471529636, 'Player': 0.2808824355394257, 'Director': 0.2012549174947853, 'Scout': 0.14763478806961922, 'Club': 0.20318341445028992, 'Offer': 0.1254959967663124, 'ScoutingAssignment': 0.21678038572140362, 'ScoutingReport': 0.25267279289563177, 'PlayerProfile': 0.2904123917938123, 'PlayerAttribute': 0.30076061345532484, 'TargetPosition': 0.7366842964305249, 'LongList': 0.22149131081650525, 'HeadScout': 0.3165436764150311, 'ShortList': 0.25704129978623275}, 'Recommendation': {'Recommendation': 0.9467610253735672, 'TeamSportsScoutingSystem': 0.28423185844541937, 'Employee': 0.15602987508633503, 'Player': 0.21064163184993834, 'Director': 0.09961178185162307, 'Scout': 0.16360857301047652, 'Club': 0.21349687448651072, 'Offer': 0.24517642622027988, 'ScoutingAssignment': 0.19996121414834356, 'ScoutingReport': 0.22723861441753263, 'PlayerProfile': 0.2903410213572166, 'Player

[('TSS', None),
 ('abstract Person', 'Employee'),
 ('Player', 'Player'),
 ('HeadCoach', 'HeadScout'),
 ('Director', 'Director'),
 ('Scout', 'Scout'),
 ('Club', 'Club'),
 ('Offer', 'Offer'),
 ('ScoutingAssignmnet', 'ScoutingAssignment'),
 ('ScoutReport', 'ScoutingReport'),
 ('PlayerProfile', 'PlayerProfile'),
 ('PlayerAttribute', 'PlayerAttribute'),
 (None, 'TeamSportsScoutingSystem'),
 (None, 'TargetPosition'),
 (None, 'LongList'),
 (None, 'ShortList')]

In [4646]:
mapping = enum_mapping[0] + cls_mapping[0]

for map in mapping:
    if map[0] and map[1]:
        ref_cls.cls_atr[map[0]]["score"] = 1
        ref_cls.cls_atr[map[0]]["counterpart"] = map[1]

        stu_cls.cls_atr[map[1]]["score"] = 1
        stu_cls.cls_atr[map[1]]["counterpart"] = map[0]

        if ref_cls.cls_atr[map[0]]["type"] != stu_cls.cls_atr[map[1]]["type"]:
            # type mismatch
            ref_cls.cls_atr[map[0]]["score"] = 0.5
            stu_cls.cls_atr[map[1]]["score"] = 0.5

In [4647]:
ref_cls.cls_atr

{'Position': {'score': 0,
  'type': 'enum',
  'dsl': 'Position(GK, LB)',
  'counterpart': None,
  'attributes': {'GK': {'score': 0, 'counterpart': None},
   'LB': {'score': 0, 'counterpart': None}}},
 'Recommendation': {'score': 1,
  'type': 'enum',
  'dsl': 'Recommendation(KEY_PLAYER, FIRST_TEAM_PLAYER, RESERVE_TEAM_PLAYER, PROSPECTIVE_PLAYER, NOT_RECOMMENDED)',
  'counterpart': 'Recommendation',
  'attributes': {'KEY_PLAYER': {'score': 0, 'counterpart': None},
   'FIRST_TEAM_PLAYER': {'score': 0, 'counterpart': None},
   'RESERVE_TEAM_PLAYER': {'score': 0, 'counterpart': None},
   'PROSPECTIVE_PLAYER': {'score': 0, 'counterpart': None},
   'NOT_RECOMMENDED': {'score': 0, 'counterpart': None}}},
 'ScoutingStatus': {'score': 0,
  'type': 'enum',
  'dsl': 'ScoutingStatus(LONG_LISTED, SHORT_LISTED, RECOMMENDED_FOR_SIGNING, OFFER_MADE)',
  'counterpart': None,
  'attributes': {'LONG_LISTED': {'score': 0, 'counterpart': None},
   'SHORT_LISTED': {'score': 0, 'counterpart': None},
   'RECOM

## Stage 1.2 Class Mapping (with all info)

In [4648]:
ref_classes = []
stu_classes = []

for cls in ref_cls.cls_name:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
        ref_classes.append(cls)

for cls in stu_cls.cls_name:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
        stu_classes.append(cls)

In [4649]:
# get name embedding
# all info
similarity_all = []
mde_embedding = sgram_mde
threshold = 0.7


# reuse saved embedding result...
# may cause problem
# if os.path.exists(student_dir + "ada_embedding.txt"):
#   pass

# with open(student_dir + "ada_embedding.txt", 'r') as json_file:
#   dict_sim_all = json.load(json_file)

if False:
    pass

else:
    for index, node in enumerate(ref_cls.cls_name):
        text_1 = get_all_info(
            index, ref_cls.cls_name, ref_cls.raw_dsl, ref_cls.rel.raw_dsl, True
        )
        emb_i = get_embedding(text_1)
        mde_pair = []
        for j, stu_node in enumerate(stu_cls.cls_name):
            text_2 = get_all_info(
                j, stu_cls.cls_name, stu_cls.raw_dsl, stu_cls.rel.raw_dsl, True
            )
            emb_j = get_embedding(text_2)
            mde_sim = cosine_distance(emb_i, emb_j)
            mde_pair.append(mde_sim)

        # apply third quartile
        similarity_all.append(mde_pair)

    dict_sim_all = {}
    for i in range(len(ref_cls.cls_name)):
        dict_sim_all[ref_cls.cls_name[i]] = {}
        for j in range(len(stu_cls.cls_name)):
            dict_sim_all[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_all[i][
                j
            ]

In [4650]:
with open(student_dir + "ada_embedding.txt", "w") as json_file:
    json.dump(dict_sim_all, json_file)

In [4651]:
student_dir

'results/gpt-4/sherpa/03_TSS/'

In [4652]:
ref_classes

['Position', 'ScoutingStatus', 'ScoutKind', 'TSS']

In [4653]:
stu_classes

['TeamSportsScoutingSystem', 'TargetPosition', 'LongList', 'ShortList']

In [4654]:
dict_sim_all

{'Position': {'Recommendation': 0.8079217823679182,
  'TeamSportsScoutingSystem': 0.7627924089897553,
  'Employee': 0.7735570692258937,
  'Player': 0.78079911716202,
  'Director': 0.7458105332807372,
  'Scout': 0.7661538915803264,
  'Club': 0.7945192168204267,
  'Offer': 0.7671666748692328,
  'ScoutingAssignment': 0.7608710646911971,
  'ScoutingReport': 0.7442650719172951,
  'PlayerProfile': 0.7565542817063662,
  'PlayerAttribute': 0.7587910613700283,
  'TargetPosition': 0.8072487722828308,
  'LongList': 0.7361644064633082,
  'HeadScout': 0.7665836102445983,
  'ShortList': 0.7337331464538285},
 'Recommendation': {'Recommendation': 0.9509893640672223,
  'TeamSportsScoutingSystem': 0.7919031640443507,
  'Employee': 0.7795864760540311,
  'Player': 0.7909390961486967,
  'Director': 0.750078656499594,
  'Scout': 0.7666448741912515,
  'Club': 0.7878305260469088,
  'Offer': 0.7553860553050957,
  'ScoutingAssignment': 0.7471938172486762,
  'ScoutingReport': 0.815827470774207,
  'PlayerProfile'

In [4655]:
mapping = match_classes(ref_classes, stu_classes, dict_sim_all, 0.8)
mapping[0]

{'Position': {'Recommendation': 0.8079217823679182, 'TeamSportsScoutingSystem': 0.7627924089897553, 'Employee': 0.7735570692258937, 'Player': 0.78079911716202, 'Director': 0.7458105332807372, 'Scout': 0.7661538915803264, 'Club': 0.7945192168204267, 'Offer': 0.7671666748692328, 'ScoutingAssignment': 0.7608710646911971, 'ScoutingReport': 0.7442650719172951, 'PlayerProfile': 0.7565542817063662, 'PlayerAttribute': 0.7587910613700283, 'TargetPosition': 0.8072487722828308, 'LongList': 0.7361644064633082, 'HeadScout': 0.7665836102445983, 'ShortList': 0.7337331464538285}, 'Recommendation': {'Recommendation': 0.9509893640672223, 'TeamSportsScoutingSystem': 0.7919031640443507, 'Employee': 0.7795864760540311, 'Player': 0.7909390961486967, 'Director': 0.750078656499594, 'Scout': 0.7666448741912515, 'Club': 0.7878305260469088, 'Offer': 0.7553860553050957, 'ScoutingAssignment': 0.7471938172486762, 'ScoutingReport': 0.815827470774207, 'PlayerProfile': 0.7807783450390694, 'PlayerAttribute': 0.76364127

[('Position', 'TargetPosition'),
 ('ScoutingStatus', 'TeamSportsScoutingSystem'),
 ('ScoutKind', None),
 ('TSS', None),
 (None, 'LongList'),
 (None, 'ShortList')]

In [4656]:
for map in mapping[0]:
    if map[0] and map[1]:
        ref_cls.cls_atr[map[0]]["score"] = 1
        ref_cls.cls_atr[map[0]]["counterpart"] = map[1]

        stu_cls.cls_atr[map[1]]["score"] = 1
        stu_cls.cls_atr[map[1]]["counterpart"] = map[0]

        if ref_cls.cls_atr[map[0]]["type"] != stu_cls.cls_atr[map[1]]["type"]:
            # type mismatch
            ref_cls.cls_atr[map[0]]["score"] = 0.5
            stu_cls.cls_atr[map[1]]["score"] = 0.5

## Stage 2.1 Attribute mapping

In [4657]:
def get_cosine_disntance(text1, text2, embedding):
    avg = np.sum(embedding[text1], axis=0)
    emb_i = avg / len(text1)

    avg = np.sum(embedding[text2], axis=0)
    emb_j = avg / len(text2)

    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

In [4658]:
def create_cosine_similarity_dict(attr_1, attr_2, embedding):
    # attr_1: list[list] split camel case into separated case
    # attr_2: list[list] split camel case into separated case
    # embedding:
    # raw_1: list[str] camcel case attributes
    # raw_2: list[str] camcel case attributes

    # create cosine distance between two attributes with embeeding

    # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
    similarities = []
    for att in attr_1:
        emb_i = get_mde_embedding(att, embedding)
        pair = []
        for attribute in attr_2:
            emb_j = get_mde_embedding(attribute, embedding)

            sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
            pair.append(sim)

        similarities.append(pair)

    dict_attr = {}
    for i in range(len(attr_1)):
        dict_attr[attr_1[i]] = {}
        for j in range(len(attr_2)):
            dict_attr[attr_1[i]][attr_2[j]] = similarities[i][j]

    return dict_attr

In [4659]:
def match_attributes(raw_1, raw_2, dict_attr, threshold=0.5):
    # map attributes lists raw_1 and raw_2
    # >>> mapAttributes(['deviceStatus'], ['id'], dic)[0]
    def node_subst_cost_attr(node1, node2):
        # threshod as 0.45
        if dict_attr[node1["name"]][node2["name"]] < threshold:
            return 3
        else:
            return 1 - dict_attr[node1["name"]][node2["name"]]

    G_att_1 = nx.Graph()
    G_att_2 = nx.Graph()
    for node in raw_1:
        G_att_1.add_node(node, name=node)

    for node in raw_2:
        G_att_2.add_node(node, name=node)

    for v in nx.optimize_edit_paths(
        G_att_1,
        G_att_2,
        node_subst_cost=node_subst_cost_attr,
        edge_match=None,
        timeout=20,
    ):
        minv = v
    # minv

    return minv

In [4660]:
def check_attributes_type(attr_1, attr_2, ref_dict, stu_dict):
    # given two attribute, compare whether their type can match
    attr_1 = attr_1.split()  # from reference solution do not need to check it
    attr_2 = attr_2.split()

    # if enumerate literal and attributes
    # if len(attr_1) != len(attr_2):
    #   return 0.5, 0.5

    # if an attributes use a regular class as its type
    if len(attr_2) == 2:
        t = attr_2[0].strip()
        cls = stu_dict.get(t, None)
        if cls is not None and (
            stu_dict[t]["type"] == "regular" or stu_dict[t]["type"] == "abstract"
        ):
            return 0.5, 0.5

    if len(attr_1) == 2 or len(attr_2) == 2:
        t = attr_1[0].strip()
        cls = ref_dict.get(t, None)
        # print(cls)
        t_2 = attr_2[0].strip()
        cls_2 = stu_dict.get(t_2, None)
        # print(cls_2)
        if cls is not None:
            if ref_dict[t]["counterpart"] != t_2:
                return 0.5, 0.5
        if cls_2 is not None:
            if stu_dict[t_2]["counterpart"] != t:
                return 0.5, 0.5

    return 1, 1

In [4661]:
x = check_attributes_type(
    ("CommandStatus commandStatus", "ControllerStatus status")[0],
    ("CommandStatus commandStatus", "ControllerStatus status")[1],
    ref_cls.cls_atr,
    stu_cls.cls_atr,
)
x

(1, 1)

### Stage 2.1.1 Attribute <-> Attribute (mapped classes)

In [4662]:
# map classes
pairs = []

# get matched pairs
for key in ref_cls.cls_atr:
    if ref_cls.cls_atr[key]["counterpart"] != None:
        pair = [key, ref_cls.cls_atr[key]["counterpart"]]
        pairs.append(pair)

for pair in pairs:
    if pair[0] and pair[1]:
        print(pair)

        # map attributes:
        # matched_ref = set()
        # matched_attr = set()

        raw_1 = []
        for attributes in ref_cls.cls_atr[pair[0]]["attributes"]:
            raw_1.append(attributes)

        raw_2 = []
        for attributes in stu_cls.cls_atr[pair[1]]["attributes"]:
            raw_2.append(attributes)

        cos_dict = create_cosine_similarity_dict(raw_1, raw_2, sgram_mde)
        mappings = match_attributes(raw_1, raw_2, cos_dict)[0]
        print("mapping", mappings)

        for mapping in mappings:
            if mapping[0] != None and mapping[1] != None:
                scores = check_attributes_type(
                    mapping[0], mapping[1], ref_cls.cls_atr, stu_cls.cls_atr
                )
                ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["score"] = scores[0]
                ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["counterpart"] = (
                    mapping[1],
                    pair[1],
                )
                stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["score"] = scores[1]
                stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["counterpart"] = (
                    mapping[0],
                    pair[0],
                )

        print("=" * 20)

['Position', 'TargetPosition']
mapping [('GK', None), ('LB', None), (None, 'string positionName')]
['Recommendation', 'Recommendation']
mapping [('KEY_PLAYER', 'KeyPlayer'), ('FIRST_TEAM_PLAYER', 'FirstTeamPlayer'), ('RESERVE_TEAM_PLAYER', 'ReserveTeamPlayer'), ('PROSPECTIVE_PLAYER', 'ProspectivePlayer'), ('NOT_RECOMMENDED', None), (None, 'NotAGoodSigning')]
['ScoutingStatus', 'TeamSportsScoutingSystem']
mapping [('LONG_LISTED', None), ('SHORT_LISTED', None), ('RECOMMENDED_FOR_SIGNING', None), ('OFFER_MADE', None), (None, 'string systemName')]
['abstract Person', 'Employee']
mapping [('string firstName', 'string employeeName'), ('string lastName', None)]
['Player', 'Player']
mapping [('ScoutingStatus status', None), (None, 'string playerName'), (None, 'string playerAttributes')]
['HeadCoach', 'HeadScout']
mapping [(None, 'string headScoutName')]
['Director', 'Director']
mapping [(None, 'string directorName')]
['Scout', 'Scout']
mapping [('ScoutKind scoutKind', 'string scoutName')]
['Cl

### Stage 2.1.2 Attribute <-> Attribute (between any classes)

In [4663]:
def create_cosine_similarity_list(attr_1, attr_2, embedding):
    # attr_1: list[list] split camel case into separated case
    # attr_2: list[list] split camel case into separated case
    # embedding:
    # raw_1: list[str] camcel case attributes
    # raw_2: list[str] camcel case attributes

    # create cosine distance between two attributes with embeeding

    # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
    similarities = []
    for att in attr_1:
        emb_i = get_mde_embedding(att, embedding)
        pair = []
        for attribute in attr_2:
            emb_j = get_mde_embedding(attribute, embedding)

            sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
            pair.append(sim)

        similarities.append(pair)

    return similarities

In [4664]:
def combine_two_dict(
    list_1, list_2, atr_1, atr_2, cls_1, cls_2, weight_1=0.9, weight_2=0.1
):
    # list_1: similarity between cls
    # list_2: cos similarity between attributes
    if weight_1 + weight_2 != 1:
        raise ValueError("weight_1 + weight_2 != 1")
    result = {}
    for i in range(len(atr_1)):
        atr_r = atr_1[i]
        cls_r = cls_1[i]
        tmp_ref = (atr_r, cls_r)
        result[tmp_ref] = {}
        for j in range(len(atr_2)):
            atr_s = atr_2[j]
            cls_s = cls_2[j]
            tmp_stu = (atr_s, cls_s)

            result[tmp_ref][tmp_stu] = weight_1 * list_1[i][j] + weight_2 * list_2[i][j]

    return result

In [4665]:
# attr between any classes

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
    for attributes in ref_cls.cls_atr[cls]["attributes"]:
        if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
            raw_1.append(attributes)
            ref_source.append(cls)
            tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get attributes on student sides
for cls in stu_cls.cls_atr:
    for attributes in stu_cls.cls_atr[cls]["attributes"]:
        if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
            raw_2.append(attributes)
            stu_source.append(cls)
            tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
    print(mapping)
    if mapping[0] != None and mapping[1] != None:
        scores = check_attributes_type(
            mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr
        )

        ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["score"] = min(
            scores[0], 0.5
        )
        ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (
            mapping[1]
        )

        stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["score"] = min(
            scores[1], 0.5
        )
        stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (
            mapping[0]
        )

        if ref_cls.cls_atr[mapping[0][1]]["counterpart"] == mapping[1][1]:
            ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["score"] = min(
                scores[0], 1
            )

            stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["score"] = min(
                scores[1], 1
            )
print("=" * 20)

(('GK', 'Position'), None)
(('LB', 'Position'), None)
(('NOT_RECOMMENDED', 'Recommendation'), None)
(('LONG_LISTED', 'ScoutingStatus'), None)
(('SHORT_LISTED', 'ScoutingStatus'), None)
(('RECOMMENDED_FOR_SIGNING', 'ScoutingStatus'), ('NotAGoodSigning', 'Recommendation'))
(('OFFER_MADE', 'ScoutingStatus'), None)
(('REGULAR_SCOUNT', 'ScoutKind'), ('string headScoutName', 'HeadScout'))
(('HEAD_SCOUT', 'ScoutKind'), None)
(('string lastName', 'abstract Person'), ('string systemName', 'TeamSportsScoutingSystem'))
(('ScoutingStatus status', 'Player'), None)
(('Position position', 'PlayerProfile'), ('string positionName', 'TargetPosition'))
(None, ('string playerName', 'Player'))
(None, ('string playerAttributes', 'Player'))
(None, ('string directorName', 'Director'))
(None, ('string clubName', 'Club'))
(None, ('string assignmentName', 'ScoutingAssignment'))
(None, ('string pros', 'ScoutingReport'))
(None, ('string profileName', 'PlayerProfile'))
(None, ('string longListName', 'LongList'))
(N

## Stage 2.2.1 Attribute mapping atr -> cls

In [4666]:
#  get non-mapped attributes on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
    for attributes in ref_cls.cls_atr[cls]["attributes"]:
        if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
            raw_1.append(attributes)
            ref_source.append(cls)
            tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
        raw_2.append(cls)
        stu_source.append(cls)
        tup_s.append((cls, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
    print(mapping)
    if mapping[0] != None and mapping[1] != None:
        # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["score"] = 0.5
        ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (
            None,
            mapping[1][1],
        )

        stu_cls.cls_atr[mapping[1][1]]["score"] = 0.5
        stu_cls.cls_atr[mapping[1][1]]["counterpart"] = mapping[0]

print("=" * 20)

(('GK', 'Position'), None)
(('LB', 'Position'), None)
(('NOT_RECOMMENDED', 'Recommendation'), None)
(('LONG_LISTED', 'ScoutingStatus'), None)
(('SHORT_LISTED', 'ScoutingStatus'), None)
(('OFFER_MADE', 'ScoutingStatus'), None)
(('HEAD_SCOUT', 'ScoutKind'), None)
(('ScoutingStatus status', 'Player'), None)
(None, ('LongList', 'LongList'))
(None, ('ShortList', 'ShortList'))


## Stage 2.2.2 Attribute mapping cls -> atr

In [4667]:
#  get non-mapped cls on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get class on instrucotr sides
for cls in ref_cls.cls_atr:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
        raw_1.append(cls)
        ref_source.append(cls)
        tup_r.append((cls, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    for attributes in stu_cls.cls_atr[cls]["attributes"]:
        if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
            raw_2.append(attributes)
            stu_source.append(cls)
            tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
    print(mapping)
    if mapping[0] != None and mapping[1] != None:
        # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[mapping[0][1]]["score"] = 0.5
        ref_cls.cls_atr[mapping[0][1]]["counterpart"] = mapping[1]

        stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["score"] = 0.5
        stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (
            None,
            mapping[0][1],
        )

print("=" * 20)

(('ScoutKind', 'ScoutKind'), None)
(('TSS', 'TSS'), None)
(None, ('string playerName', 'Player'))
(None, ('string playerAttributes', 'Player'))
(None, ('string directorName', 'Director'))
(None, ('string clubName', 'Club'))
(None, ('string assignmentName', 'ScoutingAssignment'))
(None, ('string pros', 'ScoutingReport'))
(None, ('string profileName', 'PlayerProfile'))
(None, ('string longListName', 'LongList'))
(None, ('string shortListName', 'ShortList'))


## Stage 3: Relationship mapping

In [4668]:
def check_relations_classes(ref_index, stu_index, ref_elements, stu_elements):
    # ref_index: index of the element in the dsl
    # stu_index: index of the element in the dsl
    # ref_elements: list[str] elements in the dsl
    # stu_elements: list[str] elements in the dsl

    ref_class_1 = ref_elements[ref_index]
    # check whether this is an abstract class
    dict_keys = list(ref_attributes.keys())
    if not ref_class_1 in dict_keys:
        abstract = "abstract " + ref_class_1
        ref_class_1 = abstract

    true_pair = ref_attributes[ref_class_1]["counterpart"]
    if isinstance(true_pair, tuple):
        boolean_1 = False
    elif true_pair == None:
        boolean_1 = False
    else:
        tmp = true_pair.replace("abstract ", "")
        boolean_1 = stu_elements[stu_index] == tmp
    return boolean_1

#### Compare rel

In [4669]:
def compare_edges(ref_e, stu_e):
    # compare if two relationships match
    # return bool, score, score

    ref_elements = ref_e.split()
    stu_elements = stu_e.split()
    # print(ref_elements, stu_elements)

    # exact match first
    # length must match
    n_ref = len(ref_elements)
    n_stu = len(stu_elements)
    if n_ref == n_stu and n_ref == 5:
        # all element must match
        boolean_1 = check_relations_classes(1, 1, ref_elements, stu_elements)

        boolean_4 = check_relations_classes(4, 4, ref_elements, stu_elements)

        # type
        boolean_2 = ref_elements[2] == stu_elements[2]

        # association
        boolean_0 = ref_elements[0] == stu_elements[0]

        # association
        boolean_3 = ref_elements[3] == stu_elements[3]

        if boolean_0 and boolean_1 and boolean_2 and boolean_3 and boolean_4:
            print("exact match success")
            print(ref_e, "|||", stu_e)
            return True, 1, 1

        boolean_1_4 = False
        boolean_4_1 = False
        boolean_1_4 = check_relations_classes(1, 4, ref_elements, stu_elements)
        boolean_4_1 = check_relations_classes(4, 1, ref_elements, stu_elements)

        # check if the relationship is fliped, this only works for associate, not containment
        if "associate" in ref_elements[2] and "associate" in stu_elements[2]:
            # association
            boolean_0 = ref_elements[0] == stu_elements[3]

            # association
            boolean_3 = ref_elements[3] == stu_elements[0]

        if boolean_0 and boolean_1_4 and boolean_3 and boolean_4_1 and boolean_2:
            print("match success, flipped associate")
            print(ref_e, "|||", stu_e)
            return True, 1, 1

        # check if relationship is partially match
        if (boolean_1 and boolean_4) or (boolean_1_4 and boolean_4_1):
            print("match partially success")
            print(ref_e, "|||", stu_e)
            return True, 0.5, 0.5

    if n_ref == n_stu and n_ref == 3:
        # all element must match
        boolean_0 = check_relations_classes(0, 0, ref_elements, stu_elements)
        boolean_2 = check_relations_classes(2, 2, ref_elements, stu_elements)

        # type
        boolean_1 = ref_elements[1] == stu_elements[1]

        if boolean_0 and boolean_1 and boolean_2:
            print("match success")
            print(ref_e, "|||", stu_e)
            # ref_dict[ref_e]["score"] = 1
            # ref_dict[ref_e]["counterpart"] = stu_e

            # stu_dict[stu_e]["score"] = 1
            # stu_dict[stu_e]["counterpart"] = ref_e
            return True, 1, 1

    return False, 0, 0

In [4670]:
for i, ref_tmp in enumerate(edges[0].raw_dsl):
    matchings = []
    for j, stu_tmp in enumerate(edges[1].raw_dsl):
        if (
            edges[0].rels[i]["counterpart"] is None
            and edges[1].rels[j]["counterpart"] is None
        ):
            result = compare_edges(ref_tmp, stu_tmp)
            if result[0]:
                matching = (i, j, result[1], result[2])
                matchings.append(matching)
                # matched_stu_edges.add(j)
                # matched_ref_edges.add(i)
                # edges[0].rels[i]["score"] = result[1]
                # edges[0].rels[i]["counterpart"] = stu_tmp

                # edges[1].rels[j]["score"] = result[2]
                # edges[1].rels[j]["counterpart"] = ref_tmp
    print(matchings)
    if len(matchings) > 0:
        print()
        highest_score = 0
        matching = None

        for m in matchings:
            if m[2] > highest_score:
                matching = m
                highest_score = m[2]
        j_j = matching[1]
        i_i = matching[0]

        edges[0].rels[i_i]["score"] = m[2]
        edges[0].rels[i_i]["counterpart"] = edges[1].raw_dsl[j_j]

        edges[1].rels[j_j]["score"] = m[3]
        edges[1].rels[j_j]["counterpart"] = edges[0].raw_dsl[i_i]

## Result: Precision/Recall/F1

### save matching to disk

In [4671]:
if not os.path.exists(student_out_dir):
    os.makedirs(student_out_dir)

In [4672]:
# with open( student_out_dir + 'matching.pkl', 'wb') as outp:
#     pickle.dump(grader, outp, pickle.HIGHEST_PROTOCOL)

In [4673]:
student_out_dir

'results/gpt-4/sherpa/03_TSS/result/'

In [4674]:
# as requested in comment

with open(student_out_dir + "solution_matching.txt", "w") as file:
    file.write(json.dumps(grader.ref.cls_atr))


with open(student_out_dir + "student_matching.txt", "w") as file:
    file.write(json.dumps(grader.stu.cls_atr))

with open(student_out_dir + "solution_matching_relationship.txt", "w") as file:
    file.write(json.dumps(grader.ref.rel.rels))


with open(student_out_dir + "student_matching_relationship.txt", "w") as file:
    file.write(json.dumps(grader.stu.rel.rels))

In [4675]:
student_out_dir

'results/gpt-4/sherpa/03_TSS/result/'

###### to local

In [4676]:
# # # be careful...

# with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.ref.cls_atr))


# with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.stu.cls_atr))

# with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.ref.rel.rels))


# with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.stu.rel.rels))

In [4677]:
algo_result = {}
algo_result["class"] = {"precision": 0, "recall": 0, "f1": 0}
algo_result["attribute"] = {"precision": 0, "recall": 0, "f1": 0}
algo_result["relationship"] = {"precision": 0, "recall": 0, "f1": 0}

### Class

In [4678]:
# recell
count = 0
recall = 0
for key in list(ref_attributes.keys()):
    count += 1
    recall += ref_attributes[key]["score"]
print(count, recall)

if count != 0:
    algo_result["class"]["recall"] = recall / count

16 12.5


In [4679]:
# precision
count = 0
precision = 0
for key in list(stu_attributes.keys()):
    count += 1
    precision += stu_attributes[key]["score"]

print(count, precision)
if count != 0:
    algo_result["class"]["precision"] = precision / count

if algo_result["class"]["recall"] + algo_result["class"]["precision"] != 0:
    algo_result["class"]["f1"] = (
        2
        * (algo_result["class"]["recall"] * algo_result["class"]["precision"])
        / (algo_result["class"]["recall"] + algo_result["class"]["precision"])
    )

16 12.5


### Attribute

In [4680]:
# recall
count = 0
recall = 0
for key in list(ref_attributes.keys()):
    attrs = ref_attributes[key]["attributes"]

    for att in attrs:
        count += 1
        recall += ref_attributes[key]["attributes"][att]["score"]
print(count, recall)

if count != 0:
    algo_result["attribute"]["recall"] = recall / count

24 13.0


In [4681]:
count = 0
precision = 0
for key in list(stu_attributes.keys()):
    attrs = stu_attributes[key]["attributes"]

    for att in attrs:
        count += 1
        precision += stu_attributes[key]["attributes"][att]["score"]
print(count, precision)

if count != 0:
    algo_result["attribute"]["precision"] = precision / count

r = algo_result["attribute"]["recall"]
p = algo_result["attribute"]["precision"]

if r + p != 0:
    algo_result["attribute"]["f1"] = 2 * (r * p) / (r + p)

25 13.0


### Relationship

In [4682]:
count = 0
recall = 0
for tmp in edges[0].rels:
    attrs = tmp["score"]

    count += 1
    recall += attrs
print(count, recall)

if count != 0:
    algo_result["relationship"]["recall"] = recall / count

0 0


In [4683]:
count = 0
precision = 0
for tmp in edges[1].rels:
    attrs = tmp["score"]

    count += 1
    precision += attrs
print(count, precision)

if count != 0:
    algo_result["relationship"]["precision"] = precision / count
r = algo_result["relationship"]["recall"]
p = algo_result["relationship"]["precision"]

if r + p != 0:
    algo_result["relationship"]["f1"] = 2 * (r * p) / (r + p)

0 0


In [4684]:
# as requested in comment

with open(student_out_dir + "algo_result.txt", "w") as file:
    file.write(json.dumps(algo_result))